In [1]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import parafac
from numpy.linalg import matrix_rank, pinv

## Choose parameters
For now, we assume that the latent variables are independent.

#### 1) Define third order tensor of noise variables

In [2]:
nlat = 3
third_moms = np.arange(1,(nlat+1))
third_moms

array([1, 2, 3])

In [3]:
Omega = np.zeros((nlat,nlat,nlat))
for i in range(nlat):
    Omega[i,i,i] = third_moms[i]

In [9]:
Omega

array([[[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 2., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 3.]]])

In [4]:
# Sample entries of G1 from Unif([0,1])
np.random.seed(101)
G1 = np.random.random_sample(size=(10,2))
G1

array([[0.51639863, 0.57066759],
       [0.02847423, 0.17152166],
       [0.68527698, 0.83389686],
       [0.30696622, 0.89361308],
       [0.72154386, 0.18993895],
       [0.55422759, 0.35213195],
       [0.1818924 , 0.78560176],
       [0.96548322, 0.23235366],
       [0.08356143, 0.60354842],
       [0.72899276, 0.27623883]])

In [5]:
np.random.seed(505)
G2 = np.random.random_sample(size=(8,2))
G2

array([[0.8506181 , 0.29254118],
       [0.22308853, 0.19771466],
       [0.37789012, 0.13853412],
       [0.01082837, 0.08256878],
       [0.95951925, 0.72198539],
       [0.06453682, 0.81941047],
       [0.77452391, 0.57546951],
       [0.17440146, 0.57344167]])

## Compute observed covariance matrix and third-order tensor

In [27]:
Sigma1 = np.matmul(G1, np.transpose(G1))
Sigma2 = np.matmul(G2, np.transpose(G2))

In [28]:
tucker_tensor1 = (Omega[:2,:2,:2],[G1,G1,G1])
T1 = tl.tucker_to_tensor(tucker_tensor1, skip_factor=None, transpose_factors=False)

In [29]:
tucker_tensor2 = (Omega[np.ix_([0,2],[0,2],[0,2])],[G2,G2,G2])
T2 = tl.tucker_to_tensor(tucker_tensor2, skip_factor=None, transpose_factors=False)

In [30]:
# This means that the first latent variable is the joint one. 

In [31]:
# Check by hand: it does the same as the following code

d1 = G1.shape[0]

Omega1 = Omega[:2,:2,:2]
T1_v2 = np.zeros((d1,d1,d1))
for i1 in range(d1):
    for i2 in range(d1):
        for i3 in range(d1):
            for j in range(2):
                T1_v2[i1,i2,i3] = T1_v2[i1,i2,i3] + Omega1[j,j,j] * G1[i1,j] * G1[i2,j] * G1[i3,j]
                
(T1_v2 == T1).all()

True

## Identify nr of joint latent variables and joint distribution

In [32]:
matrix_rank(Sigma1)

2

In [33]:
matrix_rank(Sigma2)

2

#### 1) Identify the mixing matrices of the environments (up to permutation of columns)
(Use CANDECOMP of TensorLy - can we do this symbolically?)

In [34]:
def identify_mixing_up_to_perm(Sigma, T):
    
    rk = matrix_rank(Sigma)
    weights, factors = parafac(T, rank=rk,  normalize_factors=False)
    factor = factors[0]
    
    Lambda = np.matmul(np.matmul(pinv(factor), Sigma), np.transpose(pinv(factor)))
    rescaler = np.diag(np.sqrt(np.diag(Lambda)))
    G = np.matmul(factor, rescaler)
    return G

In [35]:
G1_P1 = identify_mixing_up_to_perm(Sigma1, T1)
G1_P1

array([[ 0.57066766, -0.51639847],
       [ 0.17152166, -0.02847418],
       [ 0.83389696, -0.68527675],
       [ 0.89361312, -0.30696598],
       [ 0.18993905, -0.72154381],
       [ 0.35213203, -0.5542275 ],
       [ 0.78560179, -0.18189219],
       [ 0.23235379, -0.96548316],
       [ 0.60354843, -0.08356127],
       [ 0.27623893, -0.72899268]])

In [36]:
# Compare with true mixing matrix
G1

array([[0.51639863, 0.57066759],
       [0.02847423, 0.17152166],
       [0.68527698, 0.83389686],
       [0.30696622, 0.89361308],
       [0.72154386, 0.18993895],
       [0.55422759, 0.35213195],
       [0.1818924 , 0.78560176],
       [0.96548322, 0.23235366],
       [0.08356143, 0.60354842],
       [0.72899276, 0.27623883]])

In [37]:
G2_P2 = identify_mixing_up_to_perm(Sigma2, T2)
G2_P2

array([[-0.29255044, -0.8506095 ],
       [-0.19771709, -0.22308271],
       [-0.13853824, -0.37788604],
       [-0.0825689 , -0.01082594],
       [-0.72199584, -0.95949802],
       [-0.81941117, -0.06451273],
       [-0.57547794, -0.77450698],
       [-0.57344357, -0.1743846 ]])

In [38]:
# Compare with true mixing matrix
G2

array([[0.8506181 , 0.29254118],
       [0.22308853, 0.19771466],
       [0.37789012, 0.13853412],
       [0.01082837, 0.08256878],
       [0.95951925, 0.72198539],
       [0.06453682, 0.81941047],
       [0.77452391, 0.57546951],
       [0.17440146, 0.57344167]])

#### 2) Identify the third moments of the noise variables (up to permutation)

In [39]:
def diag_tensor_to_array(T):
    d = np.zeros(T.shape[0])
    for i in range(T.shape[0]):
        d[i] = T[i,i,i]
    return d

def identify_third_mom_up_to_perm(G, T):
    pinv_G = pinv(G)
    Omega_r = (T,[pinv_G,pinv_G,pinv_G])
    Omega_r = tl.tucker_to_tensor(Omega_r, skip_factor=None, transpose_factors=False)
    third_moms = diag_tensor_to_array(Omega_r).round(5)
    return third_moms

In [40]:
third_moms1 = identify_third_mom_up_to_perm(G1_P1,T1)
third_moms1

array([ 2., -1.])

In [41]:
third_moms2 = identify_third_mom_up_to_perm(G2_P2,T2)
third_moms2

array([-3., -1.])

#### 3) Identify joint latent space

In [42]:
joint_abs_moms = np.array(list(set(abs(third_moms1)).intersection(set(abs(third_moms2)))))
joint_abs_moms

# joint latent space is 1-dimensional and the corresponding noise variable has absolute third-order moment equl to 2

array([1.])

In [43]:
# Reorder columns of the mixing matrix such that the columns corresponding to the joint latent space come first 
# (in same order)

def reorder_columns(G,third_moms,joint_abs_moms):
    third_abs_moms = abs(third_moms)
    nr_lat = G.shape[1]
    P = np.zeros((nr_lat,nr_lat))

    positions = []
    for i, mom in enumerate(joint_abs_moms):
        current_pos = np.where(abs(third_moms)==mom)[0][0]
        positions.append(current_pos)
        P[current_pos,i]=1
    
    remaining_rows = set(np.arange(nr_lat)) - set(positions)
    
    for i, row in enumerate(remaining_rows):
        P[row,len(joint_abs_moms)+i] = 1
        
    return np.matmul(G, P)

In [50]:
G1_P1 = reorder_columns(G1_P1, third_moms1,joint_abs_moms)
G1_P1

array([[-0.51639847,  0.57066766],
       [-0.02847418,  0.17152166],
       [-0.68527675,  0.83389696],
       [-0.30696598,  0.89361312],
       [-0.72154381,  0.18993905],
       [-0.5542275 ,  0.35213203],
       [-0.18189219,  0.78560179],
       [-0.96548316,  0.23235379],
       [-0.08356127,  0.60354843],
       [-0.72899268,  0.27623893]])

In [51]:
G2_P2 = reorder_columns(G2_P2, third_moms2,joint_abs_moms)
G2_P2

array([[-0.8506095 , -0.29255044],
       [-0.22308271, -0.19771709],
       [-0.37788604, -0.13853824],
       [-0.01082594, -0.0825689 ],
       [-0.95949802, -0.72199584],
       [-0.06451273, -0.81941117],
       [-0.77450698, -0.57547794],
       [-0.1743846 , -0.57344357]])

#### 4) Define "joint mixing matrix" (up to blockwise permutation)

In [77]:
def joint_mixing_matrix(G_list, nr_joint):
    nr_domains = len(G_list)
    
    total_nr_obs = sum([G.shape[0] for G in G_list])
    total_nr_lat = sum([G.shape[1]-nr_joint for G in G_list]) + nr_joint
    G_large = np.zeros((total_nr_obs, total_nr_lat))
    
    current_row = 0
    current_col = nr_joint
    
    for G in G_list:
        nrows, ncols = G.shape
        nr_domain_specific = ncols - nr_joint
        
        G_large[np.ix_(np.arange(current_row,(current_row+nrows)),np.arange(nr_joint))] = G[:,:nr_joint]
        G_large[np.ix_(np.arange(current_row,(current_row+nrows)),np.arange(current_col,(current_col+nr_domain_specific)))] \
        = G[:,nr_joint:]
        
        current_row = current_row + nrows
        current_col = current_col + nr_domain_specific
        
    return G_large

In [78]:
joint_mixing_matrix([G1_P1, G2_P2],len(joint_abs_moms))

array([[-0.51639847,  0.57066766,  0.        ],
       [-0.02847418,  0.17152166,  0.        ],
       [-0.68527675,  0.83389696,  0.        ],
       [-0.30696598,  0.89361312,  0.        ],
       [-0.72154381,  0.18993905,  0.        ],
       [-0.5542275 ,  0.35213203,  0.        ],
       [-0.18189219,  0.78560179,  0.        ],
       [-0.96548316,  0.23235379,  0.        ],
       [-0.08356127,  0.60354843,  0.        ],
       [-0.72899268,  0.27623893,  0.        ],
       [-0.8506095 ,  0.        , -0.29255044],
       [-0.22308271,  0.        , -0.19771709],
       [-0.37788604,  0.        , -0.13853824],
       [-0.01082594,  0.        , -0.0825689 ],
       [-0.95949802,  0.        , -0.72199584],
       [-0.06451273,  0.        , -0.81941117],
       [-0.77450698,  0.        , -0.57547794],
       [-0.1743846 ,  0.        , -0.57344357]])

TODO:
- Create GitHub repository and invite Chandler
- In reorder columns: not only look on absolute value but also on sign! Match signs
- Check if code works for more joint independent factors
- Rewrite code such that it also works if joint space is only "upstream"
- Can we identify the joint latent graph if we only have ONE pure child in each environment???

## Identify joint latent graph

In [ ]:
# TO BE DONE